### Домашнее задание №13 по теме «Улучшение качества модели»

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

#### Задание
_________
__Цель:__  
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.
Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная - наличие болезни сердца (`HeartDisease`). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.
__Этапы работы:__. 

1. Получите данные и загрузите их в рабочую среду.  
2. Подготовьте датасет к обучению моделей:  
    a. Категориальные переменные переведите в цифровые значения. Можно использовать `pd.get_dummies, preprocessing.LabelEncoder.`
    Старайтесь не использовать для этой задачи циклы.
3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.  
4. Обучите модель логистической регрессии с параметрами по умолчанию.  
5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:  
   `cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])`
6. Оптимизируйте 3-4 параметра модели:  
    a. Используйте `GridSearchCV.`  
    b. Используйте `RandomizedSearchCV.`  
    c. \*Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.  
    d. Повторите п. 5 после каждого итогового изменения параметров.  
7. Сформулируйте выводы по проделанной работе:  
    a. Сравните метрики построенных моделей.  
    b. \*Сравните с полученными результатами в домашнем задании по теме __«Ансамблирование».__
____
Для получения зачета по этому домашнему заданию минимально необходимо:
1. обучить одну модель классификации;
2. оптимизировать параметры, используя метод из п. 6a; 
3. вывести значения метрик. 

#### 1. Загружаем данные

In [2]:
heart_raw = pd.read_csv('Downloads/Heart.csv')

In [3]:
heart = heart_raw.copy()

#### 2. Предобработка

In [4]:
heart['Sex'] = heart['Sex'].map({'M': 0, 'F': 1})
heart['ExerciseAngina'] = heart['ExerciseAngina'].map({'N': 0, 'Y': 1})

In [5]:
heart = pd.get_dummies(data=heart, columns=heart.select_dtypes('object').columns).copy()

In [6]:
heart.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,0,140,289,0,172,0,0.000,0,0,1,0,0,0,1,0,0,0,1
1,49,1,160,180,0,156,0,1.000,1,0,0,1,0,0,1,0,0,1,0
2,37,0,130,283,0,98,0,0.000,0,0,1,0,0,0,0,1,0,0,1
3,48,1,138,214,0,108,1,1.500,1,1,0,0,0,0,1,0,0,1,0
4,54,0,150,195,0,122,0,0.000,0,0,0,1,0,0,1,0,0,0,1


#### 3. Разделение выборки

In [7]:
X,y = heart.drop(columns='HeartDisease', axis=1),heart['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((734, 18), (184, 18))

#### 4&5 Обучение логистической регрессии с параметрами по умолчанию и подсчет метрик

In [8]:
res = pd.DataFrame()
scoring=['accuracy', 'recall', 'precision','f1']

In [9]:
def get_metrics(df, model = LogisticRegression(), model_name='LogisticRegression_default'):
    X,y = df.drop(columns='HeartDisease', axis=1),df['HeartDisease']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = model
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    rep_train = cross_validate(model,X_train, y_train, cv=10, scoring=scoring)
    rep_train = pd.DataFrame(rep_train)
    s = pd.Series()
    s['model'] = model_name
    s = s.append(rep_train.mean())
    res = pd.DataFrame(s)
    return res

In [10]:
res_0 = get_metrics(heart)
res = pd.concat([res, res_0], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884


#### 6. Оптимизируйте 3-4 параметра модели

##### a. GridSearchCV

In [11]:
model_lr = LogisticRegression()

In [12]:
param_grid_lr = {
    'penalty': ['l1', 'l2'], 
    'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
    'l1_ratio': np.linspace(0,1,10)
              }

In [13]:
grid_GS = GridSearchCV(model_lr, param_grid_lr, cv=5, scoring='accuracy')

In [14]:
grid_GS.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy')

In [15]:
grid_GS.best_params_

{'l1_ratio': 0.0, 'penalty': 'l1', 'solver': 'liblinear'}

In [16]:
grid_GS.best_score_

0.8678408349641227

##### b. RandomizedSearchCV

In [17]:
grid_RS = RandomizedSearchCV(model_lr, param_grid_lr, cv=5, scoring='accuracy')

grid_RS.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions={'l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear']},
                   scoring='accuracy')

In [18]:
grid_RS.best_params_

{'solver': 'liblinear', 'penalty': 'l1', 'l1_ratio': 0.8888888888888888}

In [19]:
grid_RS.best_score_

0.8678408349641227

In [20]:
res_1 = get_metrics(heart, LogisticRegression(
    l1_ratio = 0.0, 
    penalty = 'l1', 
    solver='liblinear'), model_name='LogisticRegression_GS_opt' )
res = pd.concat([res, res_1], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884
LogisticRegression_GS_opt,0.005,0.002,0.868,0.895,0.869,0.881


In [21]:
res_2 = get_metrics(heart, LogisticRegression(
    l1_ratio = 0.5555555555555556, 
    penalty = 'l1', 
    solver='liblinear'), model_name='LogisticRegression_RS_opt' )
res = pd.concat([res, res_2], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884
LogisticRegression_GS_opt,0.005,0.002,0.868,0.895,0.869,0.881
LogisticRegression_RS_opt,0.005,0.002,0.868,0.895,0.869,0.881


##### c. *Добавьте 2-5 моделей классификации и вариации их параметров.

In [22]:
models=[
    {'name':'RF',
     'model': RandomForestClassifier(), 'params':{
         'n_estimators':[10,25,50,100,150,200], 
          'criterion':['gini', 'entropy'], 
          'max_depth':[3,5,7,9,11], 
          'max_samples': np.linspace(0,1,10),
          'random_state': [1,10,42]}},
    {'name':'DT',
     'model': DecisionTreeClassifier(), 'params':{
          'criterion':['gini', 'entropy'], 
          'max_depth':[3,5,7,9,11], 
          'random_state': [1,10,42]}},
    {'name':'GB',
     'model': GradientBoostingClassifier(), 'params':{
          'max_features':list(range(1,30)),
          'n_estimators':[10,25,50,100,150,200], 
          'min_samples_leaf':[1,2,3,5], 
          'random_state': [1,10,42]}},

]

res_RS=[]
for v in  models:
    res_RS.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10, random_state=1).fit(X_train, y_train)))

In [23]:
for r in res_RS:
    print(r[0], round(r[1].best_score_, 3), r[1].best_params_)

RF 0.868 {'random_state': 10, 'n_estimators': 150, 'max_samples': 1.0, 'max_depth': 9, 'criterion': 'gini'}
DT 0.843 {'random_state': 42, 'max_depth': 3, 'criterion': 'entropy'}
GB 0.872 {'random_state': 10, 'n_estimators': 200, 'min_samples_leaf': 1, 'max_features': 16}


In [24]:
res_3 = get_metrics(
    heart, 
    RandomForestClassifier(
        random_state=10,
        n_estimators = 150, 
        max_samples = 1.,
        max_depth = 9, 
        criterion='gini'
    ), model_name='RandomForestClassifier_RS_opt')
res = pd.concat([res, res_3], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884
LogisticRegression_GS_opt,0.005,0.002,0.868,0.895,0.869,0.881
LogisticRegression_RS_opt,0.005,0.002,0.868,0.895,0.869,0.881
RandomForestClassifier_RS_opt,0.142,0.009,0.868,0.908,0.861,0.883


In [25]:
res_4 = get_metrics(
    heart, 
    DecisionTreeClassifier(
        random_state=42,
        max_depth = 3, 
        criterion = 'entropy'
    ), model_name='DecisionTreeClassifier_RS_opt')
res = pd.concat([res, res_4], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884
LogisticRegression_GS_opt,0.005,0.002,0.868,0.895,0.869,0.881
LogisticRegression_RS_opt,0.005,0.002,0.868,0.895,0.869,0.881
RandomForestClassifier_RS_opt,0.142,0.009,0.868,0.908,0.861,0.883
DecisionTreeClassifier_RS_opt,0.002,0.002,0.843,0.885,0.839,0.861


In [26]:
res_5 = get_metrics(
    heart, 
    GradientBoostingClassifier(
        random_state=10,
        n_estimators = 200, 
        min_samples_leaf = 1, 
        max_features=16
    ), model_name='GradientBoostingClassifier_RS_opt')
res = pd.concat([res, res_5], axis=1)
res.T.set_index('model')

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
model,,,,,,
LogisticRegression_default,0.021,0.002,0.870,0.898,0.871,0.884
LogisticRegression_GS_opt,0.005,0.002,0.868,0.895,0.869,0.881
LogisticRegression_RS_opt,0.005,0.002,0.868,0.895,0.869,0.881
RandomForestClassifier_RS_opt,0.142,0.009,0.868,0.908,0.861,0.883
DecisionTreeClassifier_RS_opt,0.002,0.002,0.843,0.885,0.839,0.861
GradientBoostingClassifier_RS_opt,0.178,0.002,0.872,0.893,0.877,0.883


#### 7. Выводы:
_____
В целом с помощью `GridSearchCV` и `RandomizedSearchCV` можно упорядочить поиск гиперпараметров, но все равно возможные варианты перебрать нет возможности и в связи с перегрузкой вычислительных мощностей

__\*Сравнение с полученными результатами в домашнем задании по теме «Ансамблирование».__

In [27]:
with open('Downloads/dz_8.09_result.pkl', 'rb') as f:
    dz_09_result = pickle.load(f)

In [28]:
dz_09_result = pd.DataFrame(dz_09_result).T

In [29]:
dz_09_result

,duration,accuracy_train,accuracy_test,f1_train,f1_test
dtc_min_leaf=6,0.0051 sec,0.910,0.837,0.917,0.853
rfc_max_feat=6_max_sampl=0.6,0.0632 sec,0.973,0.891,0.975,0.906
rfc_default,0.1295 sec,1.000,0.897,1.000,0.910
baggcl_max_feat=6_max_sampl=0.6,0.0621 sec,0.896,0.886,0.908,0.902
stack_svc,2.2529 sec,0.963,0.897,0.967,0.912
stack_baggcl,0.9188 sec,0.936,0.880,0.942,0.896
gbc_max_feat=10_min_leaf=5,0.0224 sec,0.891,0.891,0.903,0.905


В __дз№9__ лучшей моделью оказалась `GradientBoostingClassifier` в таблице `gbc_max_feat=10_min_leaf=5` , где параметры были подобраны вручную и без кроссвалидации: 
* max_features=10, 
* n_estimators=25, 
* min_samples_leaf=5, 
* random_state=42   
 
Модель показывала высокие метрики `accuracy` и `f1` на тренировочной и тестовой выборках.  
____   
В этой дз для модели `GradientBoostingClassifier` `RandomizedSearchCV` предложил параметры:  
* max_features=16,
* n_estimators=200,
* min_samples_leaf=1, 
* random_state=10, 

Метрики `accuracy` и `f1` на тренировочной выборке ощутимо ниже.

##### Лично у меня есть сомнения на счет этих подборщиков, либо что-то не то делаю. Напишите пожалуйста коммент в отзыве
